In [ ]:
%matplotlib inline
import pickle
import matplotlib.pyplot as plt
import numpy as np
import time
import umap 
from tensorflow.python.client import device_lib
#print(device_lib.list_local_devices())
import keras
from keras import backend as K
from keras.models import Sequential
from keras.layers.core import Dense, Activation
from keras.utils import np_utils
from keras.layers import Convolution3D, Convolution2D, MaxPooling2D
from keras.layers.normalization import BatchNormalization

from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.layers import Dense, Dropout, Flatten


In [ ]:
def load_data():
    
    train_imgs_np = np.array(np.load('train_imgs'))
    train_labs_np = np.array(np.load('train_labs'))
    test_imgs_np = np.array(np.load('test_imgs'))
    test_labs_np = np.array(np.load('test_labs'))
    
    print('done loading arrays')
    
    return (train_imgs_np, train_labs_np, test_imgs_np, test_labs_np)

%time train_imgs_np, train_labs_np, test_imgs_np, test_labs_np = load_data()


In [ ]:
print(np.shape(train_imgs_np),np.shape(test_imgs_np))
print('total images: ', np.shape(train_imgs_np)[0]+np.shape(test_imgs_np)[0])

In [ ]:
file = open('trans', 'rb')
%time trans = pickle.load(file)

In [ ]:
%time embedding = trans.transform(train_imgs_np) 
#%time trans_test = trans.transform(test_imgs_np) 

In [ ]:
batch_size = 64
nz, nx, ny = 3, 224, 224
input_shape = (nx, ny, nz)
#train_imgs = train_imgs_np.reshape((8887,nx,ny,nz))
#test_imgs = test_imgs_np.reshape((2221,nx,ny,nz))

print(np.shape(train_imgs),np.shape(train_imgs_np))
#print(np.shape(embedding))
num_classes = 3

model = Sequential()

model.add(Convolution2D(32, (3, 3), input_shape=input_shape))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.3))

model.add(Convolution2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.5))

model.add(Convolution2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.7))

model.add(Flatten())  # this converts our 3D feature maps to 1D feature vectors
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

#adam uses a learning rate of 0.001
learning_rate = 0.001
optimizer = keras.optimizers.Adam()
adagrad = keras.optimizers.Adagrad(lr=learning_rate)

model.compile(loss=keras.losses.sparse_categorical_crossentropy,
    optimizer = optimizer, metrics=['accuracy'])
#metrics=how many images are correctly classified

model.summary()



In [ ]:
# Define the neural network model (Using a similar topology to parametric t-SNE)
model = Sequential()  #more detailed above
model.add(Dense(500, activation='relu', input_shape=(train_imgs_np.shape[1],)))
model.add(Dense(500, activation='relu'))
model.add(Dense(2000, activation='relu'))
model.add(Dense(2))
model.compile(loss='mean_squared_error', optimizer='Adam')


In [ ]:
# Fit the training samples
#print(type(train_imgs),type(embedding),type(test_imgs))
print(np.shape(train_imgs_np),np.shape(train_labs_np))
%time model.fit(train_imgs, train_labs_np, batch_size=32, shuffle=False, epochs=100)
#test_loss, test_acc = model.evaluate(test_imgs, test_labs_np)


In [ ]:
filename = 'finalized_model.sav'
pickle.dump(model, open(filename, 'wb'))

In [ ]:
# load the model from disk
loaded_model = pickle.load(open(filename, 'rb'))

In [ ]:
# Predict the embedding for the test samples
%time predictions = loaded_model.predict_classes(test_imgs) #predict_proba #predict_classes
print(np.shape(predictions), predictions[0])

In [ ]:
for i in range(25):
    plt.subplot(5,5,i+1)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    plt.imshow(test_imgs[i])
    plt.xlabel(class_names[test_labs_np[i]])


In [ ]:
def plot_image(i, predictions_array, true_label, img):
    class_names = ['aggs', 'columns','junk']
    predictions_array, true_label, img = predictions_array[i], true_label[i], img[i]
    plt.grid(False)
    plt.xticks([])
    plt.yticks([])

    plt.imshow(img)

    predicted_label = np.argmax(predictions_array)
     
    if predicted_label == true_label:
        #/home/vprzybylo/agg_model/CPI_ML
        #OR
        #/home/ksulia/Machine_Learning/CPI_ML/
        
        #SAVE img[i] to FILE
        #imsave('gradient.png', x)
        
        color = 'blue'
    else:
        color = 'red'

    plt.xlabel("{} {:2.0f}% ({})".format(class_names[predicted_label],
                                100*np.max(predictions_array),
                                class_names[true_label]),
                                color=color)

In [ ]:
def plot_value_array(i, predictions_array, true_label):
    predictions_array, true_label = predictions_array[i], true_label[i]
    plt.grid(False)
    plt.xticks([])
    plt.yticks([])
    
    thisplot = plt.bar(range(3), predictions_array, color="#777777")
    plt.ylim([0, 1])
    predicted_label = np.argmax(predictions_array)

    thisplot[predicted_label].set_color('red')
    thisplot[true_label].set_color('blue')

In [ ]:
print(np.shape(train_imgs_np))
class_names = ['aggs', 'columns','junk']

plt.figure(figsize=(5,5))
plt.imshow(train_imgs[20])
'''
for i in range(25):
    plt.subplot(5,5,i+1)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    plt.imshow(train_imgs[i])
    plt.xlabel(class_names[train_labs_np[i]])
'''
plt.show()

In [ ]:
# Plot the first X test images, their predicted labels, and the true labels.
# Color correct predictions in blue and incorrect predictions in red.
num_rows = 6
num_cols = 6
num_images = num_rows*num_cols

plt.figure(figsize=(2*2*num_cols, 2*num_rows))
for i in range(num_images):
    plt.subplot(num_rows, 2*num_cols, 2*i+1)
    plot_image(i, predictions, test_labs_np, test_imgs)
    plt.subplot(num_rows, 2*num_cols, 2*i+2)
    plot_value_array(i, predictions, test_labs_np)
plt.show()

In [ ]:
fig = plt.figure(figsize=(8, 16))
plt.subplot(2, 1, 1)
plt.scatter(embedding[:,0], embedding[:,1], marker='o', s=1, edgecolor='', c=train_labs_np)
plt.title("UMAP embedding - train samples")
plt.subplot(2, 1, 2)
plt.scatter(embedding_test[:,0], embedding_test[:,1], marker='o', s=1, edgecolor='', c=test_labs_np)
plt.title("UMAP embedding through neural network - test samples")
fig.tight_layout()

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

mdl = KNeighborsClassifier(n_neighbors=15, n_jobs=-1)
%time mdl.fit(embedding, train_labs_np)
%time y_pred = mdl.predict(embedding_test)


In [ ]:
from sklearn.metrics import accuracy_score

print("Accuracy: {}".format(accuracy_score(test_labs_np, y_pred)))